# SNEWS Firedrill Notebook

Last Updated: 25/04/2023


## Installing Stuff

- [**SNEWS_PT Repo**](https://github.com/SNEWS2/SNEWS_Publishing_Tools) <br>
- [**hop repo**](https://github.com/scimma/hop-client) <br>
- [**hop auth**](https://my.hop.scimma.org/hopauth/)


Start by making sure your hop credentials are updated for firedrills:

snews.alert-firedrill
snews.experiments-firedrill
snews.mma-firedrill

- Make sure you are using the latest `snews_pt` version (as of April 2023, 1.3.2, with hop 0.8.0)

```pip install -U snews-pt``` <br>
You can check your snews_pt (hop) version with `snews_pt --version` (`hop --version`)

# What is in this firedrill tutorial

This notebook has the following;<br>
- Subscribe methods for alerts, interactive notebooks and command line
- Publish observation messages, interactive notebooks and command line, different features
- Publish Heartbeat messages
- Test connection and Request feedback
- FIRE DRILL


## Using the API

In [1]:
from snews_pt.snews_pt_utils import set_name                       # to set experiment name
# from snews_pt.snews_pub import SNEWSTiersPublisher                 # to publish observations & heartbeats
from snews_pt.messages import SNEWSMessageBuilder
from snews_pt.snews_sub import Subscriber                          # to listen alerts
from snews_pt.remote_commands import test_connection, get_feedback # to test connection and get feedback
from datetime import  datetime
from time import sleep

The experiment can set their detector name. Once this is set, the software no longer raises warnings. 

In [2]:
set_name()

You are XENONnT


You can subscribe for the alerts in a jupyter notebook, however, since this opens a stream and continuosly listen, you can not use the other cells without killing it. Therefore, we recommend to subscribe using a bash terminal via the following command
```bash
user:~$ snews_pt subscribe
```

In [3]:
# this runs persistently
Subscriber(firedrill_mode=True).subscribe(outputfolder='./')

You are subscribing to ALERT
Broker:kafka://kafka.scimma.org/snews.alert-firedrill
Done


## Create and Publish Observation messages

In [4]:
observation_time = "2023-06-14T12:45:45.100000"
msg = SNEWSMessageBuilder(neutrino_time=observation_time, is_test=True, firedrill_mode=True)

What if there is an error in your message?

In [5]:
observation_time2 = "2023-06-9999999:45:100000"
msg2 = SNEWSMessageBuilder(neutrino_time=observation_time2, is_test=True, firedrill_mode=True)

ValueError: Invalid isoformat string: '2023-06-9999999:45:100000'

#### SNEWSMessageBuilder
The `SNEWSMessageBuilder` builds and validates messages for you from your given keys.

The `neutrino_time` is the most important argument. The Builder decides that this message is intended for the `CoincidenceTier` based on this input. <br>

Try calling the function with no argument -> `SNEWSMessageBuilder()` <br>
This returns `"SNEWSMessageBuilder: No messages have been built"` <br>

Another example would be the heartbeat messages <br>
If you call it with a detector status i.e. `SNEWSMessageBuilder(detector_status='ON')` this should create a message for the heartbeats. What happens if you pass both neutrino times and detector status? Try it out!

----

Few more tips;<br>
if you pass `neutrino_time` the Publisher labels it as **"Coincidence Tier"** <br>
if you pass `p_values` and `t_bin_width` it is labeled as **"Significance Tier Message"** <br>
if you pass `timing_series` it is labeled as **"Time Tier"**<br>
if you pass `retract_latest` (e.g. retract_latest=1 retracts the last 1 message) it labels as **"Retraction Message"**<br>
if you pass `detector_status` it labels as **"Heartbeat Message"** (Notice HB doesn't require time as it stamps itself)

---

The `is_test` argument is needed so that coincidence algorithm knows we are testing and ignores even if the times that are passed illogical (e.g. from past or from far future). If this is not passed, Publisher will tell you that this message is not valid.

In [9]:
## Here try and play with different arguments
# SNEWSMessageBuilder()

Notice that you only **create** the message and not send it anywhere yet! <br>
Once created, you can still display and modify the messages. See the example below.

In [12]:
msg = SNEWSMessageBuilder(neutrino_time=observation_time, is_test=True, firedrill_mode=True, detector_status='ON')
msg.selected_tiers

['SNEWSCoincidenceTierMessage', 'SNEWSHeartbeatMessage']

In [14]:
# see the selected tiers
print(msg.selected_tiers)
# see the message data
msg.messages

['SNEWSCoincidenceTierMessage', 'SNEWSHeartbeatMessage']


[SNEWSCoincidenceTierMessage
 ---------------------------
             _id : TEST_CoincidenceTier_2023-07-31T09:15:14.352389
  schema_version : 1.3.0
   detector_name : TEST
   neutrino_time : 2023-06-14T12:45:45.100000
 ---------------------------
 detector_status : ON
         is_test : True
  firedrill_mode : True
 ,
 SNEWSHeartbeatMessage
 ---------------------
             _id : TEST_Heartbeat_2023-07-31T09:15:14.363811
  schema_version : 1.3.0
   detector_name : TEST
 detector_status : ON
 ---------------------
   neutrino_time : 2023-06-14T12:45:45.100000
         is_test : True
  firedrill_mode : True
 ]

In [15]:
# the first message that is created
msg.messages[0]

SNEWSCoincidenceTierMessage
---------------------------
            _id : TEST_CoincidenceTier_2023-07-31T09:15:14.352389
 schema_version : 1.3.0
  detector_name : TEST
  neutrino_time : 2023-06-14T12:45:45.100000
---------------------------
detector_status : ON
        is_test : True
 firedrill_mode : True


In [17]:
# the second message that is created
msg.messages[1]

SNEWSHeartbeatMessage
---------------------
            _id : TEST_Heartbeat_2023-07-31T09:15:14.363811
 schema_version : 1.3.0
  detector_name : TEST
detector_status : ON
---------------------
  neutrino_time : 2023-06-14T12:45:45.100000
        is_test : True
 firedrill_mode : True


One can see that there are some more information added automatically. Now let's **send these messages to snews**

Before sending the message, let's test to see if we have a connection to the server.

In [21]:
test_connection(firedrill=True)


> Testing your connection.
> Sending to kafka://kafka.scimma.org/snews.experiments-firedrill
> Expecting from kafka://kafka.scimma.org/snews.connection-testing. 
> Going to wait 8 seconds before checking for confirmation...

	Waited for 8 sec and checked from LATEST, couldn't get a confirmation
	Maybe increase timeout and try again.


False

There were no connection! This is because at the moment, server runs the coincidence logic on the non-firedrill broker, and we were trying to test our connection to the firedrill! <br>
```bash
> Sending to kafka://kafka.scimma.org/snews.experiments-firedrill
```
Let's check to see if we have a connection to the other broker.

In [22]:
from snews_pt.remote_commands import test_connection
test_connection(firedrill=False, patience=8)


> Testing your connection.
> Sending to kafka://kafka.scimma.org/snews.experiments-test
> Expecting from kafka://kafka.scimma.org/snews.connection-testing. 
> Going to wait 8 seconds before checking for confirmation...

You (XENONnT) have a connection to the server at 2023-07-31T09:16:54.159806


True

In [25]:
msg.send_messages(firedrill_mode=False)

----------------------------------------------------------------
Sending message to CoincidenceTier on kafka://kafka.scimma.org/snews.experiments-test
_id                :TEST_CoincidenceTier_2023-07-31T09:15:14.352389
schema_version     :1.3.0
detector_name      :TEST
machine_time       :None
neutrino_time      :2023-06-14T12:45:45.100000
p_val              :None
is_test            :True
firedrill_mode     :True
sent_time          :2023-07-31T09:17:37.096038
----------------------------------------------------------------
Sending message to Heartbeat on kafka://kafka.scimma.org/snews.experiments-test
_id                :TEST_Heartbeat_2023-07-31T09:15:14.363811
schema_version     :1.3.0
detector_name      :TEST
machine_time       :None
detector_status    :ON
is_test            :True
firedrill_mode     :True
sent_time          :2023-07-31T09:17:37.097456


**Another trick** <br>
If you think that you have some other information that you think is relevant, you can still pass any other meta data by passing them as key-word arguments.

In [26]:
msg2 = SNEWSMessageBuilder(neutrino_time=observation_time, is_test=True, comment='We were taking calibration data')
msg2.messages[0], msg2.messages[0].meta

(SNEWSCoincidenceTierMessage
 ---------------------------
             _id : TEST_CoincidenceTier_2023-07-31T09:18:01.695375
  schema_version : 1.3.0
   detector_name : TEST
   neutrino_time : 2023-06-14T12:45:45.100000
 ---------------------------
         is_test : True
         comment : We were taking calibration data
 ,
 {'is_test': True, 'comment': 'We were taking calibration data'})

### JSON messages and Observation Publication via CLI

Similarly, these observation messages can also be sent with the Command Line Interface (CLI). <br>
This requires file(s) that are properly constructed and formatted as a json file then these file(s) can be passed from the terminal <br>

For this assume we have the `"observation_file.json"` which contains
```JSON
{
    "machine_time": 
        "2023-02-06T10:18:44.948273",
    "neutrino_time": 
        "2023-02-06T10:18:44.948273",
    "p_val":
        0.07,
    "something extra":
        "Our neutrino time is defined as the first event above bg"
}
```

```bash 
user:~$ snews_pt observation_file.json
```

In [46]:
from snews_pt import messages
import importlib
importlib.reload(messages)
from datetime import  datetime

test_time = datetime.utcnow().isoformat()

In [48]:
json_message = messages.SNEWSMessageBuilder.from_json("./snews_pt/test/example_coincidence_tier_message.json")
# json_message.messages
# json_message.send_messages()

## Heartbeats

SNEWS tracks several time informations with different meanings and purposes; <br>
- `neutrino_time` : this is used for searching coincidences and refers to exact time of the first neutrino detection <br>
- `machine_time` : this is an optional argument to indicate the time labeled by the machine, ideally for the observation message this would be the same as the neutrino time. Howeever, for the heartbeat messages it should be the time that detector heartbeat was checked. 
- `sent_time` : this is added by the `SNEWSMessageBuilder().send_messages()` upon execution. We use this information for latency measurements. <br>

On the server side, we also label messages by their `received_time` which allows us (for example for the heartbeats) to calculate the latency between the time detector checks their status, the time they actually send it, and the time we receive it at the server.  

In [49]:
# simply create a message for the heartbeat
msg_hb = SNEWSMessageBuilder(detector_status="ON")

Initially, the `sent_time` is set to current time i.e. same as `machine_time` however, when the message is sent, it is overwritten with the exact time at the time of execution.

In [50]:
msg_hb.messages[0]

SNEWSHeartbeatMessage
---------------------
            _id : TEST_Heartbeat_2023-07-31T09:23:50.092086
 schema_version : 1.3.0
  detector_name : TEST
detector_status : ON
---------------------
        is_test : False


In [51]:
msg_hb.send_messages()

----------------------------------------------------------------
Sending message to Heartbeat on kafka://kafka.scimma.org/snews.experiments-firedrill
_id                :TEST_Heartbeat_2023-07-31T09:23:50.092086
schema_version     :1.3.0
detector_name      :TEST
machine_time       :None
detector_status    :ON
is_test            :False
sent_time          :2023-07-31T09:23:57.784270


### Heartbeat CLI 

```bash 
user:~$ snews_pt heartbeat --status ON --time "2023-02-06T10:40:59.675226"
```
where the `--time` refers to `machine_time` and is optional. (By default it is set equal to `sent_time` which is stamped upon execution).

## Other features

We try to provide some more tools that are convenient for the user such as testing the connection and requesting feedback on their last 24 hour heartbeat status. <br>

The `test_connection` function injects a message into the coincidence stream with your detector name and current time, and labels it with `"status":"sending"`.<br>
Server sees that this is a test message and sends the confirmation by duplicating the message and replacing the status field with `"status":"received"`. It then, sends this back on the connection-test broker, where your script reads and compares the message contents.<br>
Once the same message with "received" status is found, it confirms the connection.

In [ ]:
test_connection()

### Request Feedback

At the server we keep the heartbeats in several different files. At any given time, the records of the last 24 hours is kept. Every day, the heartbeats of a given day is also stored separately and deleted after 7 days. <br>
Authorized users can request feedback on their heartbeats within the last 24hours. This selects the data belonging to your experiment and computes some statistics regarding the status, arrival times and latencies and sends an email to your requested address(es) **only if** those emails are already registered by us under that experiment.   <br>
This way, we prevent non-member and non-authorized people from accessing the heartbeat information of another experiment.

In [52]:
get_feedback(detector_name="XENONnT", email_address="kara@kit.edu", firedrill=False)

Heartbeat Feedback is requested! Expect an email from us!


## Bonus, integrating your custom scripts

If you have a follow-up script which uses the alert content and does more calculations, you can plug this script to subscribe function
```bash
snews_pt subscribe -p mycustomscript.py
```

> The `mycustomscript.py` should contain 
> ```python
> import sys, json
> data = json.load(open(sys.argv[1]))
> ``` 
> then the `data` contains the alert content as a dictionary object.

---
# Fire Drill

Now let's run a fire drill together. 
- 1. Test your connection
- 2. Open a separate notebook and start sending heartbeats

In [53]:
from snews_pt.remote_commands import test_connection
from snews_pt.messages import SNEWSMessageBuilder

In [ ]:
test_connection()

In [24]:
import time
for i in range(7):
    time.sleep(2)
    msg = SNEWSMessageBuilder(detector_status='ON', firedrill_mode=False) 
    msg.send_messages()

Message Generated for Heartbeat
----------------------------------------------------------------
Sending message to Heartbeat on kafka://kafka.scimma.org/snews.experiments-test
_id                :19_Heartbeat_2023-06-02T15:14:18.602056
detector_name      :XENONnT
machine_time       :2023-06-02T15:14:18.602056
detector_status    :ON
meta               :
schema_version     :1.3.0
sent_time          :2023-06-02T15:14:18.624390
Message Generated for Heartbeat
----------------------------------------------------------------
Sending message to Heartbeat on kafka://kafka.scimma.org/snews.experiments-test
_id                :19_Heartbeat_2023-06-02T15:14:24.462710
detector_name      :XENONnT
machine_time       :2023-06-02T15:14:24.462710
detector_status    :ON
meta               :
schema_version     :1.3.0
sent_time          :2023-06-02T15:14:24.483085
Message Generated for Heartbeat
----------------------------------------------------------------
Sending message to Heartbeat on kafka://kafka

In [22]:
from snews_pt.remote_commands import reset_cache

In [24]:
reset_cache(firedrill=False)

> Requesting to Reset the cache. If you have rights, cache will be reset


- 3. Create an observation message for your detector based on the times given below

```json
{"JUNO": {"neutrino_time": "2023-02-07T12:00:00.000000"}}, 
{"HK": {"neutrino_time": "2023-02-07T12:00:00.001076"}}, 
{"KL": {"neutrino_time": "2023-02-07T12:00:00.001090"}}, 
{"SK": {"neutrino_time": "2023-02-07T12:00:00.001090"}}, 
{"PandaX-4T": {"neutrino_time": "2023-02-07T12:00:00.002390"}}, 
{"Baksan": {"neutrino_time": "2023-02-07T12:00:00.018629"}}, 
{"IC": {"neutrino_time": "2023-02-07T12:00:00.023424"}}, 
{"LVD": {"neutrino_time": "2023-02-07T12:00:00.026127"}}, 
{"DS-20K": {"neutrino_time": "2023-02-07T12:00:00.026127"},
{"Borexino":{"neutrino_time": "2023-02-07T12:00:00.026127"}}, 
{"XENONnT": {"neutrino_time": "2023-02-07T12:00:00.026127"}}
{"KM3": {"neutrino_time": "2023-02-07T12:00:00.026338"}},
{"NOvA": {"neutrino_time": "2023-02-07T12:00:00.028417"}}, 
{"HALO-1kT": {"neutrino_time": "2023-02-07T12:00:00.030543"}}, 
{"SNOP": {"neutrino_time": "2023-02-07T12:00:00.030543"}}, 
{"SBND": {"neutrino_time": "2023-02-07T12:00:00.030548"}}, 
{"DUNE": {"neutrino_time": "2023-02-07T12:00:00.030548"}}, 
{"LZ": {"neutrino_time": "2023-02-07T12:00:00.030548"}}, 
{"MicroBooNe": {"neutrino_time": "2023-02-07T12:00:00.030548"}}
```

These times should point to the candidate star Alf Ori at <br>RA: 88.79292 deg<br>
DEC: 7.40706 deg

In [27]:
msg = SNEWSMessageBuilder(neutrino_time="2023-02-07T12:00:00.026127", is_test=True, firedrill_mode=True)

Message Generated for CoincidenceTier


In [28]:
msg.send_messages()

----------------------------------------------------------------
Sending message to CoincidenceTier on kafka://kafka.scimma.org/snews.experiments-firedrill
_id                :19_CoincidenceTier_2023-06-02T15:21:10.414950
detector_name      :XENONnT
machine_time       :2023-06-02T15:21:10.414950
neutrino_time      :2023-02-07T12:00:00.026127
p_val              :None
meta               :
	is_test            :True
schema_version     :1.3.0
sent_time          :2023-06-02T15:21:12.108085


- 4. On a separate notebook or a terminal subscribe to the alerts
```bash
 user:~$ snews_pt subscribe
```
or 

```python
from snews_pt.snews_sub import Subscriber
Subscriber().subscribe()
```

- 5. send your observation to snews